# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [33]:
weather_data = pd.read_csv('../weather_data.csv')
weather_data.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,yellowknife,62.4560,-114.3525,17.6,49,75,13.80,CA,1619315431
1,ushuaia,-54.8000,-68.3000,50.0,50,20,2.30,AR,1619315713
2,mataura,-46.1927,168.8643,45.0,82,38,8.99,NZ,1619315176
3,albany,42.6001,-73.9662,55.0,38,100,1.92,US,1619315499
4,seoul,37.5683,126.9778,66.2,45,0,13.80,KR,1619315439


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [30]:
figure_layout = {'width': '600px', 'height': '440px'}
fig = gmaps.figure(layout=figure_layout)
heatmap_layer=gmaps.heatmap_layer(weather_data[['Lat', 'Lng']], weights=weather_data['Humidity'], 
                                  max_intensity=100, point_radius=9)
heatmap_layer.opacity = .5
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='440px', width='600px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [67]:
#Enter a few weather criteria to get ideal weather locations
hotel_df = weather_data[weather_data['Max Temp'] > 85]
hotel_df = weather_data[weather_data['Humidity'] < 80]
hotel_df = weather_data[weather_data['Cloudiness'] < 30]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,ushuaia,-54.8000,-68.3000,50.00,50,20,2.30,AR,1619315713
4,seoul,37.5683,126.9778,66.20,45,0,13.80,KR,1619315439
5,panzhihua,26.5851,101.7128,80.71,22,23,0.13,CN,1619315888
6,gisborne,-38.6533,178.0042,68.00,45,5,1.99,NZ,1619315888
9,plaisance-du-touch,43.5657,1.2975,55.00,66,0,9.22,FR,1619315643
...,...,...,...,...,...,...,...,...,...
533,coihueco,-36.6167,-71.8333,57.20,72,0,5.75,CL,1619316104
534,comodoro rivadavia,-45.8667,-67.5000,53.60,82,0,7.16,AR,1619315792
539,tocopilla,-22.0920,-70.1979,65.17,75,9,5.32,CL,1619315897
545,sorong,-0.8833,131.2500,84.13,70,16,5.68,ID,1619315795


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [68]:
hotel_df.insert(9, "Hotel Name", "")
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,ushuaia,-54.8000,-68.3000,50.00,50,20,2.30,AR,1619315713,
4,seoul,37.5683,126.9778,66.20,45,0,13.80,KR,1619315439,
5,panzhihua,26.5851,101.7128,80.71,22,23,0.13,CN,1619315888,
6,gisborne,-38.6533,178.0042,68.00,45,5,1.99,NZ,1619315888,
9,plaisance-du-touch,43.5657,1.2975,55.00,66,0,9.22,FR,1619315643,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
figure_layout = {'width': '600px', 'height': '440px'}
fig = gmaps.figure(layout=figure_layout)
heatmap_layer=gmaps.heatmap_layer(weather_data[['Lat', 'Lng']], weights=weather_data['Humidity'], 
                                  max_intensity=100, point_radius=9)
heatmap_layer.opacity = .5
fig.add_layer(heatmap_layer)
fig

# Display figure
